In [1]:
#load libraries

import gurobipy as gp
import numpy as np
import pandas as pd

In [2]:
# Define initial dataframe

ROIs = pd.read_csv('ROI_data.csv')
ROIs = ROIs.set_index('Platform')

In [3]:
roi_mat = pd.read_csv('roi_mat.csv')
roi_mat = roi_mat.set_index('Unnamed: 0')

# 3. Formulate the marketing budget allocation problem as a linear program. Use gurobi to find the optimal budget allocation.

In [4]:
# Define constants

budget = 10
columns = ROIs.columns
num_dv = len(columns)
single_max = 3
ratio = 2
social_media_list = [columns.get_loc(platform) for platform in ['Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter']]

In [5]:
ROIs

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
Platform,,,,,,,,,,
ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [6]:
# Find optimal solution for initial consulting firm's estimates

model1 = gp.Model()
model1X = model1.addMVar(num_dv, ub = single_max)

model1.setObjective(gp.quicksum(ROIs.loc['ROI'][i]*model1X[i] for i in range(num_dv)), sense=gp.GRB.MAXIMIZE)

conlist1 = [0]*3
conlist1[0] = model1.addConstr((model1X[columns.get_loc('Print')] + model1X[columns.get_loc('TV')]) <= (model1X[columns.get_loc('Facebook')] + model1X[columns.get_loc('Email')]))
conlist1[1] = model1.addConstr(gp.quicksum(model1X[i] for i in social_media_list) >= ratio * (model1X[columns.get_loc('SEO')] + model1X[columns.get_loc('AdWords')]))
conlist1[2] = model1.addConstr(gp.quicksum(model1X[i] for i in range(num_dv)) <= budget)


Restricted license - for non-production use only - expires 2024-10-28


In [7]:
model1.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0xcac47b03
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e-01   1.700000e+01   0.000000e+00      0s
       1    4.5600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.560000000e-01


In [8]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [9]:
model1.objVal

0.45600000000000007

In [10]:
# Find optimal solution for second consulting firm's estimates

model2 = gp.Model()
model2X = model2.addMVar(num_dv, ub = single_max)

model2.setObjective(gp.quicksum(ROIs.loc['Second Firms ROI Estimate'][i]*model2X[i] for i in range(num_dv)), sense=gp.GRB.MAXIMIZE)

conlist2 = [0]*3
conlist2[0] = model2.addConstr((model2X[columns.get_loc('Print')] + model2X[columns.get_loc('TV')]) <= (model2X[columns.get_loc('Facebook')] + model2X[columns.get_loc('Email')]))
conlist2[1] = model2.addConstr(gp.quicksum(model2X[i] for i in social_media_list) >= ratio * (model2X[columns.get_loc('SEO')] + model2X[columns.get_loc('AdWords')]))
conlist2[2] = model2.addConstr(gp.quicksum(model2X[i] for i in range(num_dv)) <= budget)


In [11]:
model2.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x2bfada38
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e-01   1.700000e+01   0.000000e+00      0s
       1    4.5600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.560000000e-01


In [12]:
model2.x

[3.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0]

In [13]:
model2.objval

0.45600000000000007

## 5a. Are the allocations the same?

In [14]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [15]:
model2.x

[3.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0]

In [16]:
model1.objVal

0.45600000000000007

In [17]:
model2.objval

0.45600000000000007

The allocations are not the same, but the objective value is.

## 5b Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)? 

In [18]:
sum(model2.x * ROIs.loc['ROI'])

0.252

In [19]:
# Difference

sum(model2.x * ROIs.loc['ROI']) - model1.ObjVal


-0.20400000000000007

By using the second allocation for the first returns, the objective value is 0.204 less than what it would have been under the first allocation.

## 5c. Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective? 

In [20]:
sum(model1.x*ROIs.loc['Second Firms ROI Estimate'])

0.264

In [21]:
sum(model1.x*ROIs.loc['Second Firms ROI Estimate']) - model2.objval

-0.19200000000000006

The first allocation is .192 lower than the second allocation if the first assumptions are correct.

The third constraint is not useful is binding and decreases the maximum possible return. 

## 6. To explore this further perform some analysis of how your optimal allocation would change based on changes in the ROI data. Use the first ROI data as your starting point. By how much could each advertising medium’s ROI increase or decrease and still result in the same optimal allocation you found in step (3)?

In [22]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [23]:
model1.SAObjLow

[-inf, 0.039, -inf, 0.033, -inf, -inf, 0.039, -inf, -inf, 0.028999999999999998]

In [24]:
model1.SAObjUp

[0.049,
 0.062,
 0.039,
 0.046,
 0.028999999999999998,
 0.039,
 inf,
 0.039,
 0.039,
 inf]

## 7. Your boss has gained permission to reinvest half of the return. For example, if the marketing obtains a 4% return in January, the budget of February will be $10M + $10M × 4% × 50% = $10.2M. The monthly ROI for next year is given in an attached csv file. The three constraints given by your boss are still in place for each month. What is the optimal allocation for each month?

In [25]:
len(columns)

10

In [26]:
budget = 10
output = []
months = len(roi_mat)
columns = roi_mat.columns
num_dv = len(columns)

for month in range(months):

    model = gp.Model()
    modelX = model.addMVar(len(columns), ub = single_max)

    model.setObjective(gp.quicksum((roi_mat.iloc[month, i])/100*modelX[i] for i in range(len(columns))), sense=gp.GRB.MAXIMIZE)

    conlist = [0]*3
    conlist[0] = model.addConstr((modelX[columns.get_loc('Print')] + modelX[columns.get_loc('TV')]) <= (modelX[columns.get_loc('Facebook')] + modelX[columns.get_loc('Email')]))
    conlist[1] = model.addConstr(gp.quicksum(modelX[i] for i in social_media_list) >= ratio * (modelX[columns.get_loc('SEO')] + modelX[columns.get_loc('AdWords')]))
    conlist[2] = model.addConstr(gp.quicksum(modelX[i] for i in range(num_dv)) <= budget)
    
    model.Params.OutputFlag = 0 
    model.optimize()
    #print(model.x)
    #print(model.objval)
    
    values = model.x
    output.append(values)

    budget += (model.objval/2)


In [27]:
output = pd.DataFrame(output)
output.columns = columns
output

output.index = list(roi_mat.index)
output

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.000000
February,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.000000
March,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.389648,0.0,3.000000,0.000000
April,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.596856,0.000000
May,1.804100,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.000000
June,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.020172,3.000000
July,1.123777,0.0,0.0,3.000000,1.123777,0.000000,3.000000,0.0,3.000000,0.000000
August,3.000000,0.0,0.0,1.827294,0.000000,0.654588,0.000000,0.0,3.000000,3.000000
September,1.362933,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.362933
October,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,0.000000,2.955475


## 8. A stable budget is defined as a monthly allocation such that for each platform the monthly change in spend is no more than $1M. Is the allocation you found stable? If it isn’t, you do not need to solve a new optimization model. Describe how my might model this?

In the for loop, you could create a variable that stores the previous allocations and create new contraints that ensure the new variables are within $1M of the previous allocation.

In [28]:
budget = 10
output = []
months = len(roi_mat)
columns = roi_mat.columns
num_dv = len(columns)

for month in range(months):

    model = gp.Model()
    modelX = model.addMVar(len(columns), ub = single_max)

    model.setObjective(gp.quicksum((roi_mat.iloc[month, i])/100*modelX[i] for i in range(len(columns))), sense=gp.GRB.MAXIMIZE)

    conlist = [0]*5
    conlist[0] = model.addConstr((modelX[columns.get_loc('Print')] + modelX[columns.get_loc('TV')]) <= (modelX[columns.get_loc('Facebook')] + modelX[columns.get_loc('Email')]))
    conlist[1] = model.addConstr(gp.quicksum(modelX[i] for i in social_media_list) >= ratio * (modelX[columns.get_loc('SEO')] + modelX[columns.get_loc('AdWords')]))
    conlist[2] = model.addConstr(gp.quicksum(modelX[i] for i in range(num_dv)) <= budget)
    
    model.Params.OutputFlag = 0 

    if month == 0:
        model.optimize()
    else:
        conlist[3] = model.addConstrs(modelX[i] - values[i] <= 1 for i in range(num_dv))
        conlist[4] = model.addConstrs(modelX[i] - values[i] >= -1 for i in range(num_dv))
        model.optimize()

    #print(model.x)
    #print(model.objval)
    
    values = model.x
    output.append(values)

    budget += (model.objval/2)


In [29]:
output = pd.DataFrame(output)
output.columns = columns
output

output.index = list(roi_mat.index)
output

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,3.000000,0.0,0.0,1.333333,0.0,0.0,2.666667,0.0,0.000000,3.000000
February,3.000000,0.0,0.0,1.728833,1.0,0.0,1.666667,0.0,0.791000,2.000000
March,2.000000,0.0,0.0,2.124327,0.0,1.0,1.457654,0.0,1.791000,2.000000
April,1.000000,0.0,0.0,2.852807,0.0,2.0,2.457654,0.0,1.247960,1.000000
May,0.659846,0.0,0.0,1.852807,0.0,1.0,3.000000,0.0,2.247960,2.000000
June,1.659846,0.0,0.0,0.852807,0.0,0.0,3.000000,0.0,2.452584,3.000000
July,1.335724,0.0,0.0,1.852807,0.0,0.0,3.000000,0.0,3.000000,2.000000
August,0.917839,0.0,0.0,2.500000,0.0,0.0,2.000000,0.0,3.000000,3.000000
September,1.917839,0.0,0.0,2.500000,0.0,1.0,1.000000,0.0,3.000000,2.238275
October,0.917839,0.0,0.0,3.000000,0.0,2.0,2.000000,0.0,2.000000,1.960641
